In [2]:
# -*- coding: utf-8 -*-
import yaml
import data
from data.read_data_temp import select_data,TrainData,file_split,DataByFile,sample_negative
from data.data_split import kfsplit
from data.datasets import prepare_loaders
from utils.wandb_init import get_run,wandb_utils
from utils.train import get_score,train_set,get_score_list
from utils.trivial import get_logger
from model.head import OriginModel
from model.trainer import train,predict
import torch
from transformers import logging

import wandb

import numpy as np
import pandas as pd 
import os
import torch

from sklearn.metrics import precision_score,recall_score,f1_score
import gc 
gc.enable()
from utils.trivial import set_seed

import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)


In [8]:
yaml_path="./config/train.yaml"
CONFIG,Logger,DEVICE=train_set(yaml_path,experimentName=None,upload=False,filename="./test/logs",is_notebook=True)

CONFIG["num_hidden_layers"]=12
CONFIG["DATA_NUM"]=5000
CONFIG["EPOCHS"]=1
CONFIG["DEBUG_MODEL"]=False
CONFIG["run_db"]=False


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/v-mxiong/.netrc


In [9]:
    

CONFIG["INPUT_DIR"]="/vc_data/users/v-mxiong/qc/QueryTitle"
with open(os.path.join(CONFIG["INPUT_DIR"],"Schema"),"r") as f:
    line=f.readlines()[0].strip()
    column_name=line.split("\t")
    CONFIG["Logger"].info(f"column_name :{column_name}")

# trainDataObj=DataByFile(CONFIG,"train")
# trainData=trainDataObj.get_data()
# CONFIG["Logger"].info("-"*10+"trainData label's value_counts"+"-"*10)
# CONFIG["Logger"].info(trainData["label"].value_counts())
# CONFIG["Logger"].info("-"*10+"trainData fold's value_counts"+"-"*10)
# CONFIG["Logger"].info(trainData["fold"].value_counts())


column_name :['Query', 'label', 'dsat']
column_name :['Query', 'label', 'dsat']


In [55]:

def read_data_file(input_file,is_debug,data_num,head=["Query","label"]):
    with open(input_file,"r",encoding="UTF-8") as input_object:
        if is_debug==True:
            input_lines=[]
            for _ in range(data_num):
                input_lines.append(input_object.readline())
        else:
            input_lines=input_object.readlines()
    input_lines=[ line.strip().split("\t") for line in input_lines]
    # may return wrong number 
    # input_lines=[ line for line in input_lines if len(line)==len(head)]
    input_lines=[ [line[0].split("$$")[0],line[1],line[2]] for line in input_lines if len(line)==len(head)]
    
    # url+title
    # input_lines=[ [" ".join(line[0].split("$$")[1:3]),line[1]] for line in input_lines if len(line)==len(head)]
    input_df=pd.DataFrame(input_lines,columns=head)
    
    return input_df


class DataByFile(object):
    def __init__(self,CONFIG,data_type,column_name=["Query","label"],iter_now=-1):
        self.CONFIG=CONFIG
        self.positivepath=os.path.join(CONFIG["INPUT_DIR"],"positivefold")
        self.negativepath=os.path.join(CONFIG["INPUT_DIR"],"negativefold")
        self.data_type=data_type
        self.column_name=column_name
        # deperate the last uncomplete negative file
        self.iter_num=self.get_negative_fold()-1

        positive_file=os.path.join(self.positivepath,data_type)
        self.positive_df=self.get_fold(read_data_file(positive_file,\
            CONFIG["DEBUG_MODEL"],data_num=CONFIG["DATA_NUM"],head=self.column_name),skip_info=False)
        self.iter_now=iter_now
    def get_data(self,select_num=None):
        return self.iter(select_num)
    def check_data_multigpu(self,df):
        world_size=self.CONFIG["world_size"]
        if self.data_type=="train":
            batch_size=self.CONFIG["TRAIN_BATCH_SIZE"]*world_size
        else:
            batch_size=self.CONFIG["DEV_BATCH_SIZE"]*world_size
            
        if len(df)%batch_size<world_size and len(df)%world_size>0:
            extra_n=world_size-(len(df)%batch_size)
            extra_df=df.sample(n=extra_n,random_state=self.CONFIG["SEED"])
            df=pd.concat([df,extra_df],axis=0)
        return df
    def get_negative_fold(self):
        info_file=os.path.join(self.negativepath,"info")
        with open(info_file,"r",encoding="UTF-8") as file_obj:
            message=file_obj.readline().strip()
            message=message.split("\t")

        if self.data_type=="train":
            return int(message[0])
        else:
            return int(message[1])

    def get_fold(self,data_df,skip_info=True):

        data_df["label"]=data_df["label"].replace(["0","1"],[0,1])
        data_df=data_df.query("label==1 or label==0")
        data_df["label"]=data_df["label"].astype(int)
        data_df.reset_index(inplace=True,drop=True)
        data_df=kfsplit(data_df,self.CONFIG,skip_info=skip_info)
        return data_df
    def iter(self,select_num):
        if select_num!=None:
            i=select_num%self.iter_num
        else:
            self.iter_now+=1
            i=(self.iter_now)%self.iter_num
        negative_file=os.path.join(self.negativepath,self.data_type)
        print(negative_file)
        print(negative_file+str(i))
        negative_df=read_data_file(negative_file+str(i),\
            self.CONFIG["DEBUG_MODEL"],data_num=self.CONFIG["DATA_NUM"],head=self.column_name)
        negative_df=self.get_fold(negative_df)
        
        i=(i+1)%self.iter_num
        train_df=pd.concat([self.positive_df,negative_df],axis=0)
        if self.CONFIG["DIST_MODEL"]:
            train_df=self.check_data_multigpu(train_df)
            
        train_df=train_df.sample(frac=1).reset_index(drop=True)

        return train_df

In [23]:
class DataByFileHard(DataByFile):
    def __init__(self,CONFIG,data_type,column_name=["Query","label"],iter_now=-1,filter_rate=1,hard_rate=0.5):

        super(DataByFileHard,self).__init__(CONFIG,data_type,column_name,iter_now)
        
        self.hardNegativeFile=os.path.join(CONFIG["INPUT_DIR"],"hardnegfold",f"hardnegative_{filter_rate}")
        self.filter_rate=filter_rate
        self.hard_rate=hard_rate
        self.hardNegativeOffset=0

        # 当前fold
        self.fold=0
        self.hard_num=int(len(self.positive_df)*self.hard_rate*(CONFIG["NUM_FOLDS"]-1)/CONFIG["NUM_FOLDS"])
        
    def get_data(self,select_num=None):
        return self.iter(select_num)
        
    def readHardNegative(self,num):

        with open(self.hardNegativeFile,"r") as f:

            size = os.fstat(f.fileno()).st_size  # 指针操作，所以无视文件大小
            f.seek(self.hardNegativeOffset)
            lines = []
            for _ in range(num):
                if f.tell()> size:
                    #从头往里加
                    self.hardNegativeOffset=0
                    f.seek(0)
                line=f.readline()
                #只需要query部分即可
                line=line.strip().split("\t")[0]

                #事实上，在第一次训练过程中已经判断过了，其实可以跳过
                # if len(line)==len(self.column_name):
                lines.append(line)

            self.hardNegativeOffset=f.tell()
    
        return lines

    def iter(self,select_num):

        if select_num!=None:
            i=select_num%self.iter_num
        else:
            self.iter_now+=1
            i=(self.iter_now)%self.iter_num
        negative_file=os.path.join(self.negativepath,self.data_type)
        negative_df=read_data_file(negative_file+str(i),\
            self.CONFIG["DEBUG_MODEL"],data_num=self.CONFIG["DATA_NUM"],head=self.column_name)
        negative_df=self.get_fold(negative_df)

        hard_query=self.readHardNegative(self.hard_num)
        replace_index=negative_df[negative_df["fold"]!=self.fold].sample(\
            n=self.hard_num,random_state=CONFIG["SEED"]).index
        negative_df.loc[replace_index,"Query"]=hard_query
        
        
        i=(i+1)%self.iter_num
        train_df=pd.concat([self.positive_df,negative_df],axis=0)
        if self.CONFIG["DIST_MODEL"]:
            train_df=self.check_data_multigpu(train_df)
            
        train_df=train_df.sample(frac=1).reset_index(drop=True)

        return train_df
    
        



In [56]:
trainDataObj1=DataByFile(CONFIG,"train",column_name=column_name)

----------data_df fold value_counts----------
----------data_df fold value_counts----------
1.0    1321268
0.0    1321268
4.0    1321267
3.0    1321267
2.0    1321267
Name: fold, dtype: int64
1.0    1321268
0.0    1321268
4.0    1321267
3.0    1321267
2.0    1321267
Name: fold, dtype: int64


In [25]:
# trainDataObj2=DataByFile(CONFIG,"train",column_name=column_name,filter_rate=0.8,hard_rate=0.5)
trainDataObj2=DataByFile(CONFIG,"test",column_name=column_name)



----------data_df fold value_counts----------
----------data_df fold value_counts----------
1.0    327213
4.0    327213
3.0    327213
2.0    327213
0.0    327213
Name: fold, dtype: int64
1.0    327213
4.0    327213
3.0    327213
2.0    327213
0.0    327213
Name: fold, dtype: int64


In [27]:
positiveTrain=trainDataObj1.positive_df
positiveTest=trainDataObj2.positive_df

In [31]:
hard_file=os.path.join(CONFIG["INPUT_DIR"],"hardnegfold",f"hardnegative_1")
hard_df=read_data_file(hard_file,False,0,head=column_name)

In [57]:
train_df=trainDataObj1.get_data()


/vc_data/users/v-mxiong/qc/QueryTitle/negativefold/train
/vc_data/users/v-mxiong/qc/QueryTitle/negativefold/train0


In [58]:
negative_df=train_df.query("label==0")

In [59]:
train_df1=trainDataObj1.get_data()
negative_df1=train_df1.query("label==0")


/vc_data/users/v-mxiong/qc/QueryTitle/negativefold/train
/vc_data/users/v-mxiong/qc/QueryTitle/negativefold/train1


In [62]:
negative_df["Query"].unique().shape

(6602994,)

In [63]:
negative_df.shape


(6603008, 4)

In [64]:
print(negative_df1["Query"].unique().shape)
print(negative_df1.shape)


(6603498,)
(6603505, 4)


In [60]:
pd.merge(left=negative_df["Query"],right=negative_df1["Query"],on="Query",how="inner")

,Query
0,estrella morente
1,*
2,7
3,7
4,1
5,1
6,1
7,1
8,1
9,1


In [38]:
val_df=train_df.query("label==0 & fold==0")

In [39]:
result=pd.merge(left=val_df["Query"],right=hard_df["Query"],on="Query",how="inner")

In [40]:
result.shape

(19, 1)

In [41]:
result

,Query
0,66
1,66
2,federalism
3,federalism
4,quiz sur les grands moments du sport
5,quiz sur les grands moments du sport
6,quiz sur les grands moments du sport
7,quiz sur les grands moments du sport
8,quiz sur les grands moments du sport
9,quiz sur les grands moments du sport


In [44]:
val_df["Query"].unique().shape

(1320701,)

In [45]:
val_df.shape

(1320701, 4)

In [46]:
hard_df["Query"].unique().shape

(75760235,)

In [47]:
hard_df["Query"].shape

(80215242,)